In [176]:
!pip install pulp
!pip install gurobipy

DEPRECATION: Loading egg at c:\users\aveli\anaconda3\lib\site-packages\pulp-2.7.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\users\aveli\anaconda3\lib\site-packages\pulp-2.7.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [177]:
import pandas as pd

In [178]:
import pulp
from pulp import LpMaximize,LpMinimize, LpProblem, LpVariable, lpSum, LpStatus, value, GUROBI

# MAINTENANCE RULES

In [179]:
# Triggered by IRI --> adjust to requirements

  # Rules
    #(1) IRI <= 4 --> routine maintenance
    #(2) 4 < IRI <= 6 --> periodical maintenance
    #(3) 6 < IRI <= 8 --> rehab, if no budget --> holding
    #(4) IRI >8 --> reconstruction, if no budget --> holding

# DATA IMPORT

## I. Supporting data

### 1. Unit price

In [180]:
# Unit price --> adjust to requirements
unit_price = {'routine': 100, 'periodical': 200, 'holding': 300, 'rehab': 600, 'reconstruction': 1000}

### 2. Deterioration rate

In [181]:
# Deterioration equation [f(IRI)], where IRI is existing IRI --> adjust to requirements

  # Conditions
    #(1) f(1)=0.2
    #(2) f(8)=0.7
    #(3) f(>8)=0.7

In [182]:
def Next_IRI(IRI):
    if 1 <= IRI <= 8:
        return IRI + 0.5/7*IRI - 0.5/7 + 0.2
    elif 8 < IRI:
        return IRI + 0.7

## II. FWP data

In [183]:
# Define file location
fp1 = 'D:/01. Work related/RENSTRA/Python test/Data/01 ACEH.xlsx'
fp2 = 'D:/01. Work related/RENSTRA/Python test/Data/28 JATIM.xlsx'
fp3 = 'D:/01. Work related/RENSTRA/Python test/Data/16 BABEL.xlsx'
sheet = 'FWP Data'

In [184]:
# Read the data into a DataFrame
df1 = pd.read_excel(fp1, sheet_name=sheet)
df2 = pd.read_excel(fp2, sheet_name=sheet)
df3 = pd.read_excel(fp3, sheet_name=sheet)

In [185]:
# Combine all dataframes
df_fwp = pd.concat([df1, df2, df3], axis=0, ignore_index=True)

# DATA PREPROCESSING

## Add prov number

In [186]:
# Rules
def get_prov(section_id):
    if len(str(section_id)) == 6:
        return str(section_id)[:1]
    elif len(str(section_id)) == 7:
        return str(section_id)[:2]
    else:
        return None

In [187]:
# Create 'prov' column
df_fwp['prov'] = df_fwp['sectionID'].apply(lambda x: get_prov(x))

## Transform IRI (lane/segment) into IRI_Avg (segment)

In [188]:
# Create a new DF in segment unit
df_segment = df_fwp.groupby(['prov','sectionID', 'locFrom', 'locTo'], as_index=False)['info_IRIAvg'].mean()

### Assign IRI class

In [189]:
# Rules
def Class(value):
    if value <= 4:
        return 'Good'
    elif 4 < value <= 6:
        return 'Fair'
    elif 6 < value <= 8:
        return 'Poor'
    else:
        return 'Very poor'

In [190]:
# Create 'Class' column
df_segment['class'] = df_segment['info_IRIAvg'].apply(Class)

## Add STA length

In [191]:
# Create 'STA length' column
df_segment['length'] = df_segment['locTo'] - df_segment['locFrom']

# MAINTENANCE ALLOCATION

## I. Assign possible budget allocation

In [192]:
# Create possible budget colums

df_segment['do_nothing'] = df_segment.apply(lambda x: x['length'] * unit_price['routine'], axis=1) #all routine
df_segment['routine'] = df_segment.apply(lambda x: x['length'] * unit_price['routine'] \
                                         if x['class'] == 'Good' else 0, axis=1)
df_segment['periodical'] = df_segment.apply(lambda x: x['length'] * unit_price['periodical'] \
                                            if x['class'] == 'Fair' else 0, axis=1)
df_segment['holding'] = df_segment.apply(lambda x: x['length'] * unit_price['holding'] \
                                         if x['class'] in ['Poor', 'Very poor'] else 0, axis=1)
df_segment['rehab'] = df_segment.apply(lambda x: x['length'] * unit_price['rehab'] \
                                       if x['class'] == 'Poor' else 0, axis=1)
df_segment['reconstruction'] = df_segment.apply(lambda x: x['length'] * unit_price['reconstruction'] \
                                                if x['class'] == 'Very poor' else 0, axis=1)

## II. Calculate total budget for each case

### Case 1: Do nothing (routine only)

In [193]:
df_segment['option1'] = df_segment['do_nothing']

### Case 2: Minimun (hold)

In [194]:
df_segment['option2'] = df_segment['routine'] + df_segment['periodical'] + \
                        df_segment['holding']

### Case 3: Unconstrained

In [195]:
df_segment['option3'] = df_segment['routine'] + df_segment['periodical'] + \
                        df_segment['rehab'] + df_segment['reconstruction']

## III. Calculate next_IRI for each case

### Case 0: Existing

In [196]:
# Create weighted_IRI column
df_segment['weighted_IRI0'] = df_segment['info_IRIAvg']*df_segment['length']

### Case 1: Do nothing

In [197]:
# Deterioration formula
def next_option1(IRI):
    if 1 <= IRI <= 8: #do nothing
        return IRI + 0.5/7*IRI - 0.5/7 + 0.2
    elif 8 < IRI: #do nothing
        return IRI + 0.7

In [198]:
# Create Next_IRI column
df_segment['option1_IRI'] = df_segment['info_IRIAvg'].map(next_option1)

In [199]:
# Create weighted_IRI column
df_segment['weighted_IRI1'] = df_segment['option1_IRI']*df_segment['length']

### Case 2: Minimum

In [200]:
# Deterioration formula
def next_option2(IRI):
    if 1 <= IRI <= 4: #routine
        return IRI + 0.5/7*IRI - 0.5/7 + 0.2
    elif 4 < IRI <=6: #periodical
        return 4
    elif 6 < IRI: #holding
        return IRI

In [201]:
# Create Next_IRI column
df_segment['option2_IRI'] = df_segment['info_IRIAvg'].map(next_option2)

In [202]:
# Create weighted_IRI column
df_segment['weighted_IRI2'] = df_segment['option2_IRI']*df_segment['length']

### Case 3: Unconstrained

In [203]:
# Deterioration formula
def next_option3(IRI):
    if 1 <= IRI <= 4: #routine
        return IRI + 0.5/7*IRI - 0.5/7 + 0.2
    elif 4 < IRI <=6: #periodical
        return 4
    elif 6 < IRI: #rehab and reconstruction
        return 3

In [204]:
# Create Next_IRI column
df_segment['option3_IRI'] = df_segment['info_IRIAvg'].map(next_option3)

In [205]:
# Create weighted_IRI column
df_segment['weighted_IRI3'] = df_segment['option3_IRI']*df_segment['length']

In [206]:
df_segment

,prov,sectionID,locFrom,locTo,info_IRIAvg,class,length,do_nothing,routine,periodical,...,option1,option2,option3,weighted_IRI0,option1_IRI,weighted_IRI1,option2_IRI,weighted_IRI2,option3_IRI,weighted_IRI3
0,1,100100,0,100,3.4950,Good,100,10000,10000,0,...,10000,10000,10000,349.50,3.873214,387.321429,3.873214,387.321429,3.873214,387.321429
1,1,100100,100,200,3.6475,Good,100,10000,10000,0,...,10000,10000,10000,364.75,4.036607,403.660714,4.036607,403.660714,4.036607,403.660714
2,1,100100,200,300,3.8950,Good,100,10000,10000,0,...,10000,10000,10000,389.50,4.301786,430.178571,4.301786,430.178571,4.301786,430.178571
3,1,100100,300,400,4.0550,Fair,100,10000,0,20000,...,10000,20000,20000,405.50,4.473214,447.321429,4.000000,400.000000,4.000000,400.000000
4,1,100100,400,500,3.5300,Good,100,10000,10000,0,...,10000,10000,10000,353.00,3.910714,391.071429,3.910714,391.071429,3.910714,391.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50872,28,2813500,29900,30000,7.8200,Poor,100,10000,0,0,...,10000,30000,60000,782.00,8.507143,850.714286,7.820000,782.000000,3.000000,300.000000
50873,28,2813500,30000,30100,3.3900,Good,100,10000,10000,0,...,10000,10000,10000,339.00,3.760714,376.071429,3.760714,376.071429,3.760714,376.071429
50874,28,2813500,30100,30200,6.1100,Poor,100,10000,0,0,...,10000,30000,60000,611.00,6.675000,667.500000,6.110000,611.000000,3.000000,300.000000
50875,28,2813500,30200,30300,4.5000,Fair,100,10000,0,20000,...,10000,20000,20000,450.00,4.950000,495.000000,4.000000,400.000000,4.000000,400.000000


# SETTING-UP BEFORE OPTIMIZATION

## I. Create new DF in section unit (Optimization by section)

In [207]:
# Aggregation conditions
agg_functions = {
    'length': 'sum',
    'do_nothing': 'sum',
    'routine': 'sum',
    'periodical': 'sum',
    'holding': 'sum',
    'rehab': 'sum',
    'reconstruction': 'sum',
    'option1': 'sum',
    'option2': 'sum',
    'option3': 'sum',
    }

In [208]:
# Create new DF
df_section = df_segment.groupby(['prov', 'sectionID']).agg(agg_functions).reset_index()

In [209]:
# Add IRI info
df_section['info_IRIAvg'] = df_segment.groupby('sectionID', as_index=False)\
                            ['weighted_IRI0'].sum()['weighted_IRI0'] /df_section['length']
df_section['option1_IRI'] = df_segment.groupby('sectionID', as_index=False)\
                            ['weighted_IRI1'].sum()['weighted_IRI1'] /df_section['length']
df_section['option2_IRI'] = df_segment.groupby('sectionID', as_index=False)\
                            ['weighted_IRI2'].sum()['weighted_IRI2'] /df_section['length']
df_section['option3_IRI'] = df_segment.groupby('sectionID', as_index=False)\
                            ['weighted_IRI3'].sum()['weighted_IRI3'] /df_section['length']

## II. Create new DF at prov level (to calculate KPI prov)

In [210]:
# Create new DF
df_prov = df_segment.groupby('prov')['length'].sum().reset_index()
df_prov = df_prov.rename(columns={'length': 'tot_length'})

## III. KPI (before optimization)

In [211]:
# KPI is the percentage of length of IRI<=6 --> adjust to requirements

### Case 0: Existing Data

In [212]:
# Filter IRI<=6
KPI_prov0 = df_segment.loc[df_segment['info_IRIAvg'] <= 6]

In [213]:
# Sum length by prov
KPI_prov0 = KPI_prov0.groupby('prov')['length'].sum().reset_index()

In [214]:
# Join total length
KPI_prov0 = pd.merge(KPI_prov0, df_prov, on='prov', how='inner')

In [215]:
# Create KPI prov
KPI_prov0['KPI'] = KPI_prov0['length']/KPI_prov0['tot_length']*100

In [216]:
# KPI at national level
KPI_nat0 = ['All', KPI_prov0['length'].sum(), KPI_prov0['tot_length'].sum(), \
            KPI_prov0['length'].sum()/KPI_prov0['tot_length'].sum()*100]

In [217]:
# Combine KPI prov and nat
KPI_prov0.loc[len(KPI_prov0)] = KPI_nat0

In [218]:
KPI_prov0

,prov,length,tot_length,KPI
0,1,1770980,2116360,83.680470
1,16,581779,601023,96.798126
2,28,1992956,2368503,84.144120
3,All,4345715,5085886,85.446567


### Case 1: Do Nothing

In [219]:
# Filter KPI
KPI_prov1 = df_segment.loc[df_segment['option1_IRI'] <= 6]

In [220]:
# Sum length by prov
KPI_prov1 = KPI_prov1.groupby('prov')['length'].sum().reset_index()

In [221]:
# Join total length
KPI_prov1 = pd.merge(KPI_prov1, df_prov, on='prov', how='inner')

In [222]:
# Create KPI prov
KPI_prov1['KPI'] = KPI_prov1['length']/KPI_prov1['tot_length']*100

In [223]:
# KPI at national level
KPI_nat1 = ['All', KPI_prov1['length'].sum(), KPI_prov1['tot_length'].sum(), \
            KPI_prov1['length'].sum()/KPI_prov1['tot_length'].sum()*100]

In [224]:
# Combine KPI prov and nat
KPI_prov1.loc[len(KPI_prov1)] = KPI_nat1

In [225]:
KPI_prov1

,prov,length,tot_length,KPI
0,1,1647140,2116360,77.828914
1,16,568929,601023,94.660105
2,28,1835705,2368503,77.504863
3,All,4051774,5085886,79.667024


### Case 2: Minimum

In [226]:
# Filter KPI
KPI_prov2 = df_segment.loc[df_segment['option2_IRI'] <= 6]

In [227]:
# Sum length by prov
KPI_prov2 = KPI_prov2.groupby('prov')['length'].sum().reset_index()

In [228]:
# Join total length
KPI_prov2 = pd.merge(KPI_prov2, df_prov, on='prov', how='inner')

In [229]:
# Create KPI prov
KPI_prov2['KPI'] = KPI_prov2['length']/KPI_prov2['tot_length']*100

In [230]:
# KPI at national level
KPI_nat2 = ['All', KPI_prov2['length'].sum(), KPI_prov2['tot_length'].sum(), \
            KPI_prov2['length'].sum()/KPI_prov2['tot_length'].sum()*100]

In [231]:
# Combine KPI prov and nat
KPI_prov2.loc[len(KPI_prov2)] = KPI_nat2

In [232]:
KPI_prov2

,prov,length,tot_length,KPI
0,1,1770980,2116360,83.680470
1,16,581779,601023,96.798126
2,28,1992956,2368503,84.144120
3,All,4345715,5085886,85.446567


### Case 3: Unconstrained

In [233]:
# Filter KPI
KPI_prov3 = df_segment.loc[df_segment['option3_IRI'] <= 6]

In [234]:
# Sum length by prov
KPI_prov3 = KPI_prov3.groupby('prov')['length'].sum().reset_index()

In [235]:
# Join total length
KPI_prov3 = pd.merge(KPI_prov3, df_prov, on='prov', how='inner')

In [236]:
# Create KPI prov
KPI_prov3['KPI'] = KPI_prov3['length']/KPI_prov3['tot_length']*100

In [237]:
# KPI at national level
KPI_nat3 = ['All', KPI_prov3['length'].sum(), KPI_prov3['tot_length'].sum(), \
            KPI_prov3['length'].sum()/KPI_prov3['tot_length'].sum()*100]

In [238]:
# Combine KPI prov and nat
KPI_prov3.loc[len(KPI_prov3)] = KPI_nat3

In [239]:
KPI_prov3

,prov,length,tot_length,KPI
0,1,2116360,2116360,100.0
1,16,601023,601023,100.0
2,28,2368503,2368503,100.0
3,All,5085886,5085886,100.0


## IV. Required costs (before optimization)

### Case 1: Do nothing

In [240]:
# Costs prov
Costs_prov1 = df_section.groupby('prov')['option1'].sum().reset_index()

In [241]:
# Costs at national level
Costs_nat1 = df_section['option1'].sum()

In [242]:
# Combine Costs prov and nat
Costs_prov1.loc[len(Costs_prov1)] = Costs_nat1
Costs_prov1.iloc[-1,0] = 'All'
Costs_prov1 = Costs_prov1.rename(columns={'option1': 'Costs'})

In [243]:
Costs_prov1

,prov,Costs
0,1,211636000
1,16,60102300
2,28,236850300
3,All,508588600


### Case 2: Minimum

In [244]:
# Costs prov
Costs_prov2 = df_section.groupby('prov')['option2'].sum().reset_index()

In [245]:
# Costs at national level
Costs_nat2 = df_section['option2'].sum()

In [246]:
# Combine Costs prov and nat
Costs_prov2.loc[len(Costs_prov2)] = Costs_nat2
Costs_prov2.iloc[-1,0] = 'All'
Costs_prov2 = Costs_prov2.rename(columns={'option2': 'Costs'})

In [247]:
Costs_prov2

,prov,Costs
0,1,348929000
1,16,75543800
2,28,400110300
3,All,824583100


### Case 3: Unconstrained

In [248]:
# Costs prov
Costs_prov3 = df_section.groupby('prov')['option3'].sum().reset_index()

In [249]:
# Costs at national level
Costs_nat3 = df_section['option3'].sum()

In [250]:
# Combine Costs prov and nat
Costs_prov3.loc[len(Costs_prov3)] = Costs_nat3
Costs_prov3.iloc[-1,0] = 'All'
Costs_prov3 = Costs_prov3.rename(columns={'option3': 'Costs'})

In [251]:
Costs_prov3

,prov,Costs
0,1,493327000
1,16,82157000
2,28,553448400
3,All,1128932400


# OPTIMIZATION

In [252]:
# Using linear programming

## I. Define constraints

In [253]:
# Constraint 1: total costs < national budget --> adjust to requirements
budget_nat = 830000000 # value must be in between Costs_nat of Case Minimum and Unconstrained

In [254]:
# Constraint 2: budget threshold due to political reasons --> adjust to requirements
budget_prov = {'prov': ['1','16','28'], 'sign':['>','>','>'], 'budget': [0,0,0]} # prov in STRING!
                # budget must be >= Costs_prov of Case Minimum. If no budget threshold, enter > 0

In [255]:
# Constraint 3: KPI threshold due to political reasons --> adjust to requirements
KPI_prov = {'prov': ['1','16','28'], 'KPI': [0,0,0]} # prov in STRING!
            # KPI must be >= KPI_prov of Case Minimum. If no KPI threshold, enter 0

## II. Optimization formula

In [256]:
def optimize_road_maintenance(df, total_budget, KPI_prov, budget_prov):
    print("Setting up the problem...")

    # Define the problem
    prob = LpProblem("RoadMaintenanceOptimization", LpMaximize)

    # Decision variables for each sectionID - Binary: 1 for option2, 0 for option3
    section_choices = LpVariable.dicts("SectionChoice", df['sectionID'].unique(), cat='Binary')

    # Objective function
    print("Configuring the objective function...")
    total_length = df['length'].sum()
    objective_terms = []
    for index, row in df.iterrows():
        term_option2 = row['length'] * (row['option2_IRI'] <= 6) * section_choices[row['sectionID']]
        term_option3 = row['length'] * (row['option3_IRI'] <= 6) * (1 - section_choices[row['sectionID']])
        objective_terms.append(term_option2 + term_option3)

    prob += lpSum(objective_terms) / total_length * 100

    # National budget constraint
    print("Applying national budget constraint...")
    national_budget_terms = []
    for index, row in df.iterrows():
        term = row['option2'] * section_choices[row['sectionID']] + row['option3'] * \
        (1 - section_choices[row['sectionID']])
        national_budget_terms.append(term)
    prob += lpSum(national_budget_terms) <= total_budget

    # Provincial budget constraints
    print("Applying provincial budget constraints...")
    for i, province in enumerate(budget_prov['prov']):
        prov_df = df[df['prov'] == province]
        prov_budget = budget_prov['budget'][i]
        provincial_budget_terms = []
        for index, row in prov_df.iterrows():
            term = row['option2'] * section_choices[row['sectionID']] + row['option3'] * \
            (1 - section_choices[row['sectionID']])
            provincial_budget_terms.append(term)
        if budget_prov['sign'][i] == '<':
            prob += lpSum(provincial_budget_terms) <= prov_budget
        elif budget_prov['sign'][i] == '>':
            prob += lpSum(provincial_budget_terms) >= prov_budget

    # Provincial KPI constraints
    print("Applying provincial KPI constraints...")
    for province in KPI_prov['prov']:
        prov_df = df[df['prov'] == province]
        prov_total_length = prov_df['length'].sum()
        prov_KPI = KPI_prov['KPI'][KPI_prov['prov'].index(province)]
        KPI_terms = []
        for index, row in prov_df.iterrows():
            term_option2 = row['length'] * (row['option2_IRI'] <= 6) * section_choices[row['sectionID']]
            term_option3 = row['length'] * (row['option3_IRI'] <= 6) * (1 - section_choices[row['sectionID']])
            KPI_terms.append(term_option2 + term_option3)
        prob += lpSum(KPI_terms) / prov_total_length * 100 >= prov_KPI

    # Solve the problem
    print("Solving the problem...")
    prob.solve(solver=GUROBI(msg=True))

    # Check the solution status and objective value
    print("Solution Status:", LpStatus[prob.status])
    print("Objective Value:", value(prob.objective))

    if LpStatus[prob.status] == 'Optimal':
        print("Optimal solution found. Extracting results...")
        results = []
        for index, row in df.iterrows():
            selected_option = 'option2' if section_choices[row['sectionID']].varValue == 1 else 'option3'
            price = row['option2'] if selected_option == 'option2' else row['option3']
            IRI = row['option2_IRI'] if selected_option == 'option2' else row['option3_IRI']
            results.append({
                "sectionID": row['sectionID'],
                "locFrom": row['locFrom'],
                "locTo": row['locTo'],
                "selected_option": selected_option,
                "price": price,
                "IRI": IRI
            })
        return pd.DataFrame(results)
    else:
        print("No optimal solution found.")
        return None

In [257]:
# Exporting outcomes
results = optimize_road_maintenance(df_segment, budget_nat, KPI_prov, budget_prov)

Setting up the problem...
Configuring the objective function...
Applying national budget constraint...
Applying provincial budget constraints...
Applying provincial KPI constraints...
Solving the problem...
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 538 columns and 1146 nonzeros
Model fingerprint: 0xd3e2a491
Variable types: 0 continuous, 538 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-03, 2e+07]
  Objective range  [1e-03, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 6e+08]
Found heuristic solution: objective -14.2622347
Found heuristic solution: objective -14.2540159
Presolve removed 6 rows and 358 columns
Presolve time: 0.02s
Presolved: 1 rows, 180 columns, 180 nonzeros
Variable types: 0 continuous, 180 integer 

In [258]:
# Merge with df_segment
df_results = pd.merge(df_segment, results, on=['sectionID', 'locFrom', 'locTo'], how='left')
df_results

,prov,sectionID,locFrom,locTo,info_IRIAvg,class,length,do_nothing,routine,periodical,...,weighted_IRI0,option1_IRI,weighted_IRI1,option2_IRI,weighted_IRI2,option3_IRI,weighted_IRI3,selected_option,price,IRI
0,1,100100,0,100,3.4950,Good,100,10000,10000,0,...,349.50,3.873214,387.321429,3.873214,387.321429,3.873214,387.321429,option2,10000,3.873214
1,1,100100,100,200,3.6475,Good,100,10000,10000,0,...,364.75,4.036607,403.660714,4.036607,403.660714,4.036607,403.660714,option2,10000,4.036607
2,1,100100,200,300,3.8950,Good,100,10000,10000,0,...,389.50,4.301786,430.178571,4.301786,430.178571,4.301786,430.178571,option2,10000,4.301786
3,1,100100,300,400,4.0550,Fair,100,10000,0,20000,...,405.50,4.473214,447.321429,4.000000,400.000000,4.000000,400.000000,option2,20000,4.000000
4,1,100100,400,500,3.5300,Good,100,10000,10000,0,...,353.00,3.910714,391.071429,3.910714,391.071429,3.910714,391.071429,option2,10000,3.910714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50872,28,2813500,29900,30000,7.8200,Poor,100,10000,0,0,...,782.00,8.507143,850.714286,7.820000,782.000000,3.000000,300.000000,option2,30000,7.820000
50873,28,2813500,30000,30100,3.3900,Good,100,10000,10000,0,...,339.00,3.760714,376.071429,3.760714,376.071429,3.760714,376.071429,option2,10000,3.760714
50874,28,2813500,30100,30200,6.1100,Poor,100,10000,0,0,...,611.00,6.675000,667.500000,6.110000,611.000000,3.000000,300.000000,option2,30000,6.110000
50875,28,2813500,30200,30300,4.5000,Fair,100,10000,0,20000,...,450.00,4.950000,495.000000,4.000000,400.000000,4.000000,400.000000,option2,20000,4.000000


## III. Results validation

### 1. Budget

In [259]:
# Costs prov
Costs_prov_opt = df_results.groupby('prov')['price'].sum().reset_index()

In [260]:
# Costs at national level
Costs_nat_opt = df_results['price'].sum()

In [261]:
# Combine Costs prov and nat
Costs_prov_opt.loc[len(Costs_prov_opt)] = Costs_nat_opt
Costs_prov_opt.iloc[-1,0] = 'All'
Costs_prov_opt = Costs_prov_opt.rename(columns={'price': 'Costs'})

In [262]:
Costs_prov_opt

,prov,Costs
0,1,349379000
1,16,76075700
2,28,404545200
3,All,829999900


### 2. KPI

In [263]:
# Filter KPI
KPI_prov_opt = df_results.loc[df_results['IRI'] <= 6]

# Sum length by prov
KPI_prov_opt = KPI_prov_opt.groupby('prov')['length'].sum().reset_index()

# Join total length
KPI_prov_opt = pd.merge(KPI_prov_opt, df_prov, on='prov', how='inner')

# Create KPI prov
KPI_prov_opt['KPI'] = KPI_prov_opt['length']/KPI_prov_opt['tot_length']*100

# KPI at national level
KPI_nat_opt = ['All', KPI_prov_opt['length'].sum(), KPI_prov_opt['tot_length'].sum(), \
            KPI_prov_opt['length'].sum()/KPI_prov_opt['tot_length'].sum()*100]

# Combine KPI prov and nat
KPI_prov_opt.loc[len(KPI_prov_opt)] = KPI_nat_opt

In [264]:
KPI_prov_opt

,prov,length,tot_length,KPI
0,1,1772480,2116360,83.751347
1,16,583552,601023,97.093123
2,28,2007739,2368503,84.768269
3,All,4363771,5085886,85.801589


In [265]:
df_results.to_excel('output.xlsx', index=False)